# Optimizing hyperparameters for DL models on the IDS2017 dataset

In this notebook, different DL models are used on the IDS2017 with hyperparameter optimization to test the performance. Deep neural networks, autoencoders, convolutional networks and RNNs are tested on the dataset.

In [1]:
from utils_ids2017 import load_ids2017, feature_selection
from notebook_utils import plot_confusion_matrix, metrics_report, calculate_metrics_by_label, test_metrics_DL, plot_overall_accuracy
from notebook_utils import test_metrics_AE
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

attack_labels = {
    0: 'BENIGN',
    7: 'FTP-Patator',
    11: 'SSH-Patator',
    6: 'DoS slowloris',
    5: 'DoS Slowhttptest',
    4: 'DoS Hulk',
    3: 'DoS GoldenEye',
    8: 'Heartbleed',
    12: 'Web Attack - Brute Force',
    14: 'Web Attack - XSS',
    13: 'Web Attack - Sql Injection',
    9: 'Infiltration',
    1: 'Bot',
    10: 'PortScan',
    2: 'DDoS'
}

## Load Dataset

In [2]:
df = load_ids2017()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 96 columns):
 #   Column                       Dtype   
---  ------                       -----   
 0   destination_port             int64   
 1   protocol                     int64   
 2   flow_duration                int64   
 3   total_fwd_packets            int64   
 4   total_backward_packets       int64   
 5   total_length_of_fwd_packets  float64 
 6   total_length_of_bwd_packets  float64 
 7   fwd_packet_length_max        float64 
 8   fwd_packet_length_min        float64 
 9   fwd_packet_length_mean       float64 
 10  fwd_packet_length_std        float64 
 11  bwd_packet_length_max        float64 
 12  bwd_packet_length_min        float64 
 13  bwd_packet_length_mean       float64 
 14  bwd_packet_length_std        float64 
 15  flow_bytes_s                 float64 
 16  flow_packets_s               float64 
 17  flow_iat_mean                float64 
 18  flow_iat_std          

In [3]:
X = df.iloc[:, 0:79]
Y = df.iloc[:, 79:]
X.info()
Y.info()
print(Y.label.value_counts())

<class 'pandas.core.frame.DataFrame'>
Index: 2827876 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   destination_port             int64  
 1   protocol                     int64  
 2   flow_duration                int64  
 3   total_fwd_packets            int64  
 4   total_backward_packets       int64  
 5   total_length_of_fwd_packets  float64
 6   total_length_of_bwd_packets  float64
 7   fwd_packet_length_max        float64
 8   fwd_packet_length_min        float64
 9   fwd_packet_length_mean       float64
 10  fwd_packet_length_std        float64
 11  bwd_packet_length_max        float64
 12  bwd_packet_length_min        float64
 13  bwd_packet_length_mean       float64
 14  bwd_packet_length_std        float64
 15  flow_bytes_s                 float64
 16  flow_packets_s               float64
 17  flow_iat_mean                float64
 18  flow_iat_std                 float64
 19  flow_

## Feature Selection

In [4]:
X = feature_selection(X, Y)

                        Feature  Information Gain
1                 flow_duration          0.226260
10               flow_packets_s          0.217878
3   total_length_of_fwd_packets          0.212005
11                flow_iat_mean          0.206657
9                  flow_bytes_s          0.203816
23                bwd_packets_s          0.199902
31       init_win_bytes_forward          0.187530
4         fwd_packet_length_max          0.185856
6        fwd_packet_length_mean          0.183422
12                 flow_iat_std          0.178729
7         bwd_packet_length_max          0.160176
32      init_win_bytes_backward          0.153665
15                bwd_iat_total          0.148547
18                  bwd_iat_max          0.147885
16                 bwd_iat_mean          0.143561
21            fwd_header_length          0.139235
5         fwd_packet_length_min          0.111322
24            min_packet_length          0.110959
13                 flow_iat_min          0.105143


## Split Dataset

The dataset is split into a training set and a testing set with a ratio of 0.8/0.2. The dataset is stratified according to the label to have an equal representation of all classes in the 2 subsets.

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y.label_code)

In [6]:
Y_train.label.value_counts()

label
BENIGN                        1817055
DoS Hulk                       184099
PortScan                       127043
DDoS                           102420
DoS GoldenEye                    8234
FTP-Patator                      6348
SSH-Patator                      4717
DoS slowloris                    4637
DoS Slowhttptest                 4399
Bot                              1565
Web Attack - Brute Force         1206
Web Attack - XSS                  522
Infiltration                       29
Web Attack - Sql Injection         17
Heartbleed                          9
Name: count, dtype: int64

In [7]:
Y_test.label.value_counts()

label
BENIGN                        454265
DoS Hulk                       46025
PortScan                       31761
DDoS                           25605
DoS GoldenEye                   2059
FTP-Patator                     1587
SSH-Patator                     1180
DoS slowloris                   1159
DoS Slowhttptest                1100
Bot                              391
Web Attack - Brute Force         301
Web Attack - XSS                 130
Infiltration                       7
Web Attack - Sql Injection         4
Heartbleed                         2
Name: count, dtype: int64

Statistics about the data

In [8]:
benign_percentage = len(Y_train.label[Y_train["label"]=="BENIGN"])/len(Y_train)
print('Percentage of benign samples: %.4f' % benign_percentage)
print(Y_train.is_attack.value_counts())

Percentage of benign samples: 0.8032
is_attack
0    1817055
1     445245
Name: count, dtype: int64


## SMOTE Resampling

In [9]:
from imblearn.over_sampling import SMOTE

def resample_dataset(X, Y, min_samples, attack_labels):
    Y = Y.drop(columns=['label'])
    combined = pd.concat([X, Y], axis=1)
    counts = Y['label_code'].value_counts()
    samples_number = {i: max(counts[i], min_samples) for i in np.unique(Y['label_code'])}
    combined_array = combined.values
    y_array = Y['label_code'].values
    resampler = SMOTE(random_state=42, sampling_strategy=samples_number)
    resampled_array, y_resampled = resampler.fit_resample(combined_array, y_array)
    X_resampled = resampled_array[:, :-Y.shape[1]]
    Y_resampled = resampled_array[:, -Y.shape[1]:]
    X_resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
    Y_resampled_df = pd.DataFrame(Y_resampled, columns=Y.columns)
    Y_resampled_df['label'] = Y_resampled_df['label_code'].map(attack_labels)
    Y_resampled_df['label'] = Y_resampled_df['label'].astype('category')
    return X_resampled_df, Y_resampled_df

X_smote_train, Y_smote_train = resample_dataset(X_train, Y_train, 100000, attack_labels)


In [10]:
Y_smote_train.label.value_counts()

label
BENIGN                        1817055
DoS Hulk                       184099
PortScan                       127043
DDoS                           102420
Bot                            100000
DoS GoldenEye                  100000
DoS Slowhttptest               100000
DoS slowloris                  100000
FTP-Patator                    100000
Heartbleed                     100000
Infiltration                   100000
SSH-Patator                    100000
Web Attack - Brute Force       100000
Web Attack - Sql Injection     100000
Web Attack - XSS               100000
Name: count, dtype: int64

In [11]:
from sklearn.preprocessing import StandardScaler

scaler_smote = StandardScaler()
scaler_smote.fit(X_smote_train)

StandardScaler()

In [12]:
# Save the model
def save_model(model, model_name):
    # Create directory if it does not exist
    model_dir = os.path.join("models", "DL_models_optimized")
    os.makedirs(model_dir, exist_ok=True)
    # Save the model
    model.save(os.path.join(model_dir, f"{model_name}.keras"))

metrics = {}

## Optimized DNN

In [ ]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Define the model builder function
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_smote_train.shape[1],)))
    
    # Experiment with different numbers of layers and units per layer
    for i in range(hp.Int('num_layers', 3, 4)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=256, step=32), activation='relu'))
        model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(1, activation='sigmoid'))
    
    # Use Adam optimizer with different learning rates
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = kt.Hyperband(build_model,
                     objective='accuracy',
                     max_epochs=20,
                     factor=3,
                     directory='optimization',
                     project_name='DNN')

# Early stopping callback
stop_early = EarlyStopping(monitor='accuracy', patience=5)

# Perform hyperparameter search
tuner.search(scaler_smote.transform(X_smote_train), Y_smote_train.is_attack, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"The optimal hyperparameters are: {best_hps.values}")

# Build the model with the optimal hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(scaler_smote.transform(X_smote_train), Y_smote_train.is_attack, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Evaluate and save the model
metrics["DNN_Optimized"] = test_metrics_DL("DNN_Optimized", model, scaler_smote, X_test, Y_test, reshape=False)
save_model(model, "DNN_SMOTE_Optimized")


Trial 10 Complete [00h 18m 38s]
accuracy: 0.9488330483436584

Best accuracy So Far: 0.9616110920906067
Total elapsed time: 05h 13m 48s

Search: Running Trial #11

Value             |Best Value So Far |Hyperparameter
3                 |4                 |num_layers
224               |192               |units_0
0.1               |0.2               |dropout_0
32                |160               |units_1
0.4               |0.1               |dropout_1
160               |160               |units_2
0                 |0                 |dropout_2
0.001             |0.0001            |learning_rate
224               |32                |units_3
0.2               |0                 |dropout_3
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/3
83266/83266 ━━━━━━━━━━━━━━━━━━━━ 2463s 30ms/step - accuracy: 0.9277 - loss: 0.1578 - 

## Optimized CNN

In [ ]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Conv1D, Flatten, Dense, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Ensure TensorFlow compatibility
tf.compat.v1.reset_default_graph()

# Define the model builder function for CNN
def build_cnn_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_smote_train.shape[1], 1)))
    
    # Tune the number of filters and kernel size
    model.add(Conv1D(filters=hp.Int('filters', min_value=32, max_value=128, step=32),
                     kernel_size=hp.Int('kernel_size', min_value=2, max_value=5, step=1),
                     activation='relu'))
    
    # Flatten layer
    model.add(Flatten())
    
    # Fully connected layers
    model.add(Dense(units=hp.Int('units', min_value=16, max_value=128, step=16), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))
    
    # Adam optimizer with different learning rates
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = kt.Hyperband(build_cnn_model,
                     objective='accuracy',
                     max_epochs=20,
                     factor=3,
                     directory='optimization',
                     project_name='cnn_tuning')

# Early stopping callback
stop_early = EarlyStopping(monitor='accuracy', patience=5)

# Perform hyperparameter search
tuner.search(scaler_smote.transform(X_smote_train).reshape(-1, X_smote_train.shape[1], 1), 
             Y_smote_train.is_attack, 
             epochs=50, 
             validation_split=0.2, 
             callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"The optimal hyperparameters are: {best_hps.values}")

# Build the model with the optimal hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(scaler_smote.transform(X_smote_train).reshape(-1, X_smote_train.shape[1], 1), 
                    Y_smote_train.is_attack, 
                    epochs=50, 
                    validation_split=0.2, 
                    callbacks=[stop_early])

# Evaluate and save the model
metrics["CNN_Optimized"] = test_metrics_DL("CNN_Optimized", model, scaler_smote, X_test, Y_test, reshape=False)
save_model(model, "CNN_SMOTE_Optimized")


## Optimized RNN

In [ ]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Define the updated balanced model builder function for RNN
def build_updated_rnn_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_smote_train.shape[1], 1)))

    # Tune the number of units in the LSTM layer
    model.add(LSTM(units=hp.Int('units', min_value=64, max_value=128, step=32)))
    
    # Fully connected layer
    model.add(Dense(units=hp.Int('dense_units', min_value=16, max_value=64, step=16), activation='relu'))

    # Dropout layer with a range from 0 to 0.5
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))
    
    # Adam optimizer with the original set of learning rates
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = kt.Hyperband(build_updated_rnn_model,
                     objective='accuracy',
                     max_epochs=20,
                     factor=3,
                     directory='optimization',
                     project_name='updated_rnn_tuning')

# Early stopping callback
stop_early = EarlyStopping(monitor='accuracy', patience=5)

# Perform hyperparameter search
tuner.search(scaler_smote.transform(X_smote_train).reshape(-1, X_smote_train.shape[1], 1), 
             Y_smote_train.is_attack, 
             epochs=50, 
             validation_split=0.2, 
             callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"The optimal hyperparameters are: {best_hps.values}")

# Build the model with the optimal hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(scaler_smote.transform(X_smote_train).reshape(-1, X_smote_train.shape[1], 1), 
                    Y_smote_train.is_attack, 
                    epochs=50, 
                    validation_split=0.2, 
                    callbacks=[stop_early])

# Evaluate and save the model
metrics["RNN_Optimized"] = test_metrics_DL("RNN_Optimized", model, scaler_smote, X_test, Y_test, reshape=False)
save_model(model, "RNN_SMOTE_Optimized")


## Autoencoder

In [ ]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Define the model builder function for Autoencoder
def build_autoencoder_model_with_threshold(hp):
    model = Sequential()
    model.add(Input(shape=(benign_data.shape[1],)))

    # Encoder
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dense(units=hp.Int('bottleneck', min_value=8, max_value=32, step=8), activation='relu'))
    
    # Decoder
    model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=128, step=32), activation='relu'))
    
    # Reconstruct the input
    model.add(Dense(benign_data.shape[1], activation='sigmoid'))

    # Compile the model
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Initialize the tuner
tuner = kt.Hyperband(build_autoencoder_model_with_threshold,
                     objective=kt.Objective("val_loss", direction="min"),
                     max_epochs=20,
                     factor=3,
                     directory='optimization',
                     project_name='autoencoder_tuning_with_threshold')

# Early stopping callback
stop_early = EarlyStopping(monitor='val_loss', patience=5)

# Perform hyperparameter search, including threshold tuning
tuner.search(scaler_AE.transform(benign_data), scaler_AE.transform(benign_data), 
             epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"The optimal hyperparameters are: {best_hps.values}")

# Build the model with the optimal hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(scaler_AE.transform(benign_data), scaler_AE.transform(benign_data), 
                    epochs=50, validation_split=0.2, callbacks=[stop_early])

# Tune the threshold separately
optimal_threshold_percentile = best_hps.Int("threshold_percentile", min_value=80, max_value=99, step=1)
metrics["AE_Optimized"], metrics_by_label = test_metrics_AE(
    "AE_Optimized", model, scaler_AE, X_test, Y_test, threshold_percentile=optimal_threshold_percentile)

# Save the model
save_model(model, "AE_SMOTE_Optimized")